In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
casas = {
    "link": [],
    "area": [],
    "quartos": [],
    "vagas": [],
    "banheiros": [],
    "local": [],
    "valor": []
}

url = 'https://www.olx.com.br'
apikey = '81977895eff2c78b50a3eee1667eeadf34e6225e'
def get_page_content(params):
    response = requests.get('https://api.zenrows.com/v1/', params=params)
    soup  = BeautifulSoup(response.text,"html.parser")
    return soup

def get_table_items(soup):
    table = soup.find("div", class_="sc-9d8733cc-0 UnonC")
    items = table.find_all('section')
    return items

page = 1
while True:
    domain = f'{url}/imoveis/aluguel/casas?o={page}'
    params = {
        'url': domain,
        'apikey': apikey,   
    }
    
    soupa = get_page_content(params)
    
    try:
        lista_items = get_table_items(soupa)
    except AttributeError:
        print("finalizamos a raspagem aparentemente com sucesso cheque o df")
        break
    
    print("--------")
    print(f"pag {page}")
    print("--------")
    
    page += 1

    for item in lista_items:
        local_atributos = item.find("ul", class_="olx-ad-card__labels-items")
        atributos = local_atributos.find_all("li", class_="olx-ad-card__albels-item") if local_atributos else []

        important = []
        for atributo in atributos:
            span_tag = atributo.find("span")
            if span_tag and "aria-label" in span_tag.attrs:
                important.append(span_tag["aria-label"])

        link_tag = item.find("a")
        casas["link"].append(link_tag['href'] if link_tag else None)

        quartos = next((x for x in important if "quartos" in x), None)
        vagas = next((x for x in important if "garagem" in x), None)
        banheiros = next((x for x in important if "banheiros" in x), None)
        area = next((x for x in important if "metros" in x), None)
        
        casas["quartos"].append(quartos)
        casas["vagas"].append(vagas)
        casas["banheiros"].append(banheiros)
        casas["area"].append(area)
        
        valor_tag = item.find("p", class_="olx-text olx-text--caption olx-text--block olx-text--regular")
        casas["valor"].append(valor_tag.text if valor_tag else None)

        local_tag = item.find("h3")
        casas["local"].append(local_tag.text if local_tag else None)
            

df = pd.DataFrame(casas)
df.drop_duplicates(inplace=True)
df.shape

In [51]:
df.to_csv('casas-alugar-brasil.csv', index=False)

In [3]:
df= pd.read_csv("casas-alugar-brasil.csv")

In [5]:
df.head(10)

,link,area,quartos,vagas,banheiros,local,valor
0,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,50 metros quadrados,2 quartos,NaN,NaN,R$ 1.000,Osasco - SP
1,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,110 metros quadrados,4 quartos,2 vagas de garagem,2 banheiros,R$ 2.200,Santo André - SP
2,https://sp.olx.com.br/vale-do-paraiba-e-litora...,70 metros quadrados,2 quartos,1 vaga de garagem,2 banheiros,R$ 1.500,Caraguatatuba - SP
3,https://ba.olx.com.br/grande-salvador/imoveis/...,27 metros quadrados,NaN,NaN,NaN,R$ 2.580,Salvador - BA
4,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,NaN,NaN,NaN,NaN,R$ 600,Itaquaquecetuba - SP
5,https://mg.olx.com.br/belo-horizonte-e-regiao/...,NaN,2 quartos,2 vagas de garagem,2 banheiros,R$ 8.000,Belo Horizonte - MG
6,https://rj.olx.com.br/rio-de-janeiro-e-regiao/...,300 metros quadrados,4 quartos,3 vagas de garagem,4 banheiros,R$ 3.500,Rio de Janeiro - RJ
7,https://pa.olx.com.br/regiao-de-belem/imoveis/...,200 metros quadrados,3 quartos,3 vagas de garagem,4 banheiros,R$ 6.800,Ananindeua - PA
8,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,125 metros quadrados,2 quartos,2 vagas de garagem,2 banheiros,R$ 3.000,São Bernardo do Campo - SP
9,https://pr.olx.com.br/regiao-de-curitiba-e-par...,350 metros quadrados,4 quartos,5 ou mais vagas de garagem,3 banheiros,R$ 6.500,Curitiba - PR


In [6]:
df.drop_duplicates(inplace=True)
df.shape

(5185, 7)

In [7]:
df.vagas[0]

nan

In [8]:
# >>>>> transoforma os campos e deixa apenas os INT

# na raspagem que fiz, vi que os dados que não apareciam eram iguais a 1
for i in df.columns[1:5]:
    df[i] = df[i].apply(lambda x: 1.0 if pd.isna(x) else float(x.split()[0]) if isinstance(x, str) else None)  
df.head()


,link,area,quartos,vagas,banheiros,local,valor
0,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,50.0,2.0,1.0,1.0,R$ 1.000,Osasco - SP
1,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,110.0,4.0,2.0,2.0,R$ 2.200,Santo André - SP
2,https://sp.olx.com.br/vale-do-paraiba-e-litora...,70.0,2.0,1.0,2.0,R$ 1.500,Caraguatatuba - SP
3,https://ba.olx.com.br/grande-salvador/imoveis/...,27.0,1.0,1.0,1.0,R$ 2.580,Salvador - BA
4,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,1.0,1.0,1.0,1.0,R$ 600,Itaquaquecetuba - SP


In [9]:
df.rename(columns={"local":"valor",
                   "valor":"local"},
          inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5185 entries, 0 to 5184
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   link       5185 non-null   object 
 1   area       5185 non-null   float64
 2   quartos    5185 non-null   float64
 3   vagas      5185 non-null   float64
 4   banheiros  5185 non-null   float64
 5   valor      5101 non-null   object 
 6   local      5185 non-null   object 
dtypes: float64(4), object(3)
memory usage: 283.7+ KB


In [9]:
df.valor[0].split()

['R$', '1.000']

In [10]:
df.valor[0].replace(".","")


'R$ 1000'

In [10]:
# >>> deixa apenas o valor 
df["valor"] = df["valor"].apply(lambda x: float(x.split()[1].replace(".","")) if isinstance(x, str) else None)

In [ ]:
df.head(20)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5185 entries, 0 to 5184
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   link       5185 non-null   object 
 1   area       5185 non-null   float64
 2   quartos    5185 non-null   float64
 3   vagas      5185 non-null   float64
 4   banheiros  5185 non-null   float64
 5   valor      5101 non-null   float64
 6   local      5185 non-null   object 
dtypes: float64(5), object(2)
memory usage: 283.7+ KB


In [47]:
!pip install geopy

Defaulting to user installation because normal site-packages is not writeable


In [12]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter # desejo fazer muitas requisições == tem q ter delay

In [16]:
casa = "ilha dos bentos - ES"
locator = Nominatim(user_agent="MyGeocoder")
# location = locator.geocode(casa)
location = geocode(casa)

In [73]:
location[1]

(-20.369746, -40.3150212)

In [15]:
geocode = RateLimiter(locator.geocode,min_delay_seconds=1)

In [17]:
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.31)

latitudes = []
longitudes = []
a = 0

for i in df.local:
    try:
        address = i
        location = geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        latitudes.append(latitude)
        longitudes.append(longitude)
    except Exception as e:
        latitudes.append(None)
        longitudes.append(None)
        print(f"Erro na linha {a}: {e}")
    
    a += 1

# Adicionar as listas de latitude e longitude ao DataFrame
df['latitude'] = latitudes
df['longitude'] = longitudes


Erro na linha 134: 'NoneType' object has no attribute 'latitude'
Erro na linha 418: 'NoneType' object has no attribute 'latitude'
Erro na linha 455: 'NoneType' object has no attribute 'latitude'
Erro na linha 552: 'NoneType' object has no attribute 'latitude'


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Rio de Janeiro -  RJ',), **{}).
Traceback (most recent call last):
  File "C:\Users\torre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\urllib3\connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\Users\torre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\urllib3\connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1776.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1776.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 331, i

Erro na linha 1782: 'NoneType' object has no attribute 'latitude'


RateLimiter caught an error, retrying (0/2 tries). Called with (*('Rio de Janeiro -  RJ',), **{}).
Traceback (most recent call last):
  File "C:\Users\torre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\urllib3\connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "C:\Users\torre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\urllib3\connection.py", line 466, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1776.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 1428, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.1776.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 331, i

Erro na linha 3956: 'NoneType' object has no attribute 'latitude'
Erro na linha 4238: 'NoneType' object has no attribute 'latitude'
Erro na linha 4632: 'NoneType' object has no attribute 'latitude'


In [18]:
df.head()

,link,area,quartos,vagas,banheiros,valor,local,latitude,longitude
0,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,50.0,2.0,1.0,1.0,1000.0,Osasco - SP,-23.532486,-46.791680
1,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,110.0,4.0,2.0,2.0,2200.0,Santo André - SP,-23.653351,-46.527904
2,https://sp.olx.com.br/vale-do-paraiba-e-litora...,70.0,2.0,1.0,2.0,1500.0,Caraguatatuba - SP,-23.620280,-45.413060
3,https://ba.olx.com.br/grande-salvador/imoveis/...,27.0,1.0,1.0,1.0,2580.0,Salvador - BA,-12.982250,-38.481277
4,https://sp.olx.com.br/sao-paulo-e-regiao/imove...,1.0,1.0,1.0,1.0,600.0,Itaquaquecetuba - SP,-23.475449,-46.351403


In [33]:
df_on = df.drop(columns=["link"])

In [34]:
df_on.head()

,area,quartos,vagas,banheiros,valor,local,latitude,longitude
0,50.0,2.0,1.0,1.0,1000.0,Osasco - SP,-23.532486,-46.791680
1,110.0,4.0,2.0,2.0,2200.0,Santo André - SP,-23.653351,-46.527904
2,70.0,2.0,1.0,2.0,1500.0,Caraguatatuba - SP,-23.620280,-45.413060
3,27.0,1.0,1.0,1.0,2580.0,Salvador - BA,-12.982250,-38.481277
4,1.0,1.0,1.0,1.0,600.0,Itaquaquecetuba - SP,-23.475449,-46.351403


In [35]:
df_on.drop(df_on[df_on["valor"] >= 40000].index,inplace=True)


In [36]:
df_on.drop(df_on[df_on["valor"] < 100].index,inplace = True)

In [37]:
df_on.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5121 entries, 0 to 5184
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   area       5121 non-null   float64
 1   quartos    5121 non-null   float64
 2   vagas      5121 non-null   float64
 3   banheiros  5121 non-null   float64
 4   valor      5037 non-null   float64
 5   local      5121 non-null   object 
 6   latitude   5113 non-null   float64
 7   longitude  5113 non-null   float64
dtypes: float64(7), object(1)
memory usage: 360.1+ KB


In [38]:
df_on.drop(df_on[df_on["valor"].isna()].index, inplace=True)

In [39]:
df_on.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5037 entries, 0 to 5184
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   area       5037 non-null   float64
 1   quartos    5037 non-null   float64
 2   vagas      5037 non-null   float64
 3   banheiros  5037 non-null   float64
 4   valor      5037 non-null   float64
 5   local      5037 non-null   object 
 6   latitude   5029 non-null   float64
 7   longitude  5029 non-null   float64
dtypes: float64(7), object(1)
memory usage: 354.2+ KB


In [26]:
df_on.describe()

,area,quartos,vagas,banheiros,valor,latitude,longitude
count,5037.000000,5037.000000,5037.000000,5037.000000,5037.000000,5029.000000,5029.000000
mean,244.021838,2.790351,2.379392,2.522732,5016.364503,-20.143966,-46.271773
std,1900.681696,1.087204,1.413088,1.448113,5593.965788,7.312371,5.182444
min,1.000000,1.000000,1.000000,1.000000,150.000000,-31.769974,-67.822078
25%,70.000000,2.000000,1.000000,1.000000,1500.000000,-23.550651,-47.810098
50%,135.000000,3.000000,2.000000,2.000000,2900.000000,-22.905639,-46.916772
75%,250.000000,3.000000,4.000000,4.000000,6500.000000,-20.464017,-45.885454
max,84000.000000,5.000000,5.000000,5.000000,39000.000000,42.846509,-0.428129


In [40]:
df_on.drop(columns=["latitude","longitude"]).corr()

ValueError: could not convert string to float: 'Osasco -  SP'

In [41]:
df_on.to_excel('casas-alugar-brasil.xlsx', index=False)

In [32]:
df.shape

(5185, 9)